# Generating metadata

### What this notebook does
**Step 1:** Create metadata file, containing a bunch of JSON-formatted trial metadata objects

**Step 2:**  Insert each trial as a record into a mongo database

This assumes that the stimuli have been uploaded to the S3 bucket using `upload_stims_to_s3.ipynb`.

In [ ]:
import os
import numpy as np
from PIL import Image
import pandas as pd
import json
import pymongo as pm
from glob import glob
from IPython.display import clear_output
import ast
import itertools
import random

In [ ]:
def list_files(paths, ext='mp4'):
    """Pass list of folders if there are stimuli in multiple folders. 
    Make sure that the containing folder is informative, as the rest of the path is ignored in naming. 
    Also returns filenames as uploaded to S3"""
    if type(paths) is not list:
        paths = [paths]
    results = []
    names = []
    for path in paths:
        results += [y for x in os.walk(path) for y in glob(os.path.join(x[0], '*.%s' % ext))]
        names += [os.path.basename(os.path.dirname(y))+'_'+os.path.split(y)[1] for x in os.walk(path) for y in glob(os.path.join(x[0], '*.%s' % ext))]
    return results,names

In [ ]:
## where are your stimulus images stored?
data_dirs = ['./pilot_dominoes'] #Where on disk are the stimuli stored?
bucket_name = 'human-physics-benchmarking-dominoes-pilot'
stim_version = 'example'
dataset_name = '{}_{}'.format(bucket_name, stim_version)
stimulus_extension = "mp4" #what's the file extension for the stims? Provide without dot

## get a list of paths to each one
full_stim_paths,filenames = list_files(data_dirs,stimulus_extension)
print('We have {} stimuli to evaluate.'.format(len(full_stim_paths)))

In [ ]:
filenames

We also want to have a number of familiarization trials. Put a couple of hand selected filenames here. The stims are expected in the S3 bucket with that filename. 

In [ ]:
familiarization_filenames = ['pilot_dominoes_SJ025_boxroom_0000_img.mp4	', 
                             'pilot_dominoes_SJ025_boxroom_0001_img.mp4	',
                             'pilot_dominoes_SJ025_boxroom_0002_img.mp4	'
                            ]

In [ ]:
## helper to build stim urls
def build_s3_url(filename, bucket_name):    
    return 'https://{}.s3.amazonaws.com/{}'.format(bucket_name, filename)

In [ ]:
## basic metadata lists
stim_urls = [build_s3_url(p,bucket_name) for p in filenames]
stim_IDs = [name.split('.')[0] for name in filenames]
familiarization_stim_urls = [build_s3_url(p,bucket_name) for p in familiarization_filenames]
familiarization_stim_IDs = [name.split('.')[0] for name in familiarization_filenames]

In [ ]:
## convert to pandas dataframe
M = pd.DataFrame([stim_IDs,stim_urls]).transpose()
M.columns = ['stim_ID', 'stim_url']
familiarization_M = pd.DataFrame([familiarization_stim_IDs,familiarization_stim_urls]).transpose()
familiarization_M.columns = ['stim_ID', 'stim_url']
# save the familiariaziation dict
familiarization_trials = familiarization_M.transpose().to_dict()
# needs to have strings as keys
familiarization_trials = {str(key):value for key, value in familiarization_trials.items()}

In [ ]:
# if needed, add code to add additional columns
pass

In [ ]:
M

In [ ]:
familiarization_M

Ok, so we now have a list of all the stimuli to evaluate. Let's create random orders of them and submit them to the database. Set the number of stimuli shown to one participant and the number of different orders to generate (`num_of_permutations`). Each of these sets can be shown to a participant, with the least often shown set being shown (so we can get away with fewer sets than subjects). 

In [ ]:
stims_per_run = len(M) #len(M) to show all stimuli
num_of_sets = 10

In [ ]:
#generate list of stimuli as dicts
L = M.to_dict(orient='records')

In [ ]:
#generate list of lists of shuffled, sampled w/o replacement stimuli
stim_sets = []
for i in range(num_of_sets):
    stim_set = random.sample(L, stims_per_run)
    stim_sets += [{str(num):stim for num,stim in enumerate(stim_set)}]

In [ ]:
len(stim_sets)

In [ ]:
stim_sets

In [ ]:
#these will be our familiarization trials
familiarization_trials

Now we have a list of sequences of stimuli.

Set up ssh bridge to write to mongodb. Insert your username. If you don't have an SSH secret set yet, run `ssh -fNL 27017:127.0.0.1:27017 USERNAME@cogtoolslab.org` in your shell.

In [ ]:
!ssh -fNL 27017:127.0.0.1:27017 fbinder@cogtoolslab.org

In [ ]:
# set vars 
auth = pd.read_csv('../auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user. Place it in the toplevel of the repo
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'cogtoolslab.org' ## cogtoolslab ip address

conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['stimuli']
coll = db[dataset_name]

In [ ]:
#⚠️drop collection if necessary. 
db.drop_collection(dataset_name) 

In [ ]:
#get list of current collections
sorted(db.list_collection_names())

In [ ]:
## actually add data now to the database
for (i,m) in enumerate(stim_sets):
    coll.insert_one({'stims':m, 'familiarization_stims': familiarization_trials})
    print('{} of {}'.format(i+1, len(J)))
    clear_output(wait=True)

print('Done inserting records into mongo! The collection name is',dataset_name)

In [ ]:
coll.estimated_document_count()

In [ ]:
coll.find_one()

In [ ]:
list(coll.find())